In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar


--2021-10-11 02:22:56--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  4.94MB/s    in 0.2s    

2021-10-11 02:22:57 (4.94 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Furniture_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [5]:
# 
total_votes_df = df.filter("total_votes >= 20")
total_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

In [6]:
new_df = total_votes_df.filter("helpful_votes/total_votes >= .50")
new_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

In [7]:
paid_vine_df = new_df.filter("vine == 'Y'")
paid_vine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35119071|R2BQOD1R0228FN|B00H2RSA88|     405483618|Sleep Innovations...|       Furniture|          3|           17|         26|   Y|                N|An okay product. ...|Three-stars is co...| 2015-08-27|
|         US|   44737123| RC31RUPFOHBHQ|B0125QZ50G|     350975212|Zinus Viscolatex ...|       Furniture|          4|    

In [8]:
unpaid_vine_df = new_df.filter("vine == 'N'")
unpaid_vine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

In [9]:
#Total number of reviews for PAID 
total_number_paid = paid_vine_df.distinct().count() 
print(total_number_paid)

136


In [11]:
#Total number of reviews for UNPAID 
total_number_unpaid = unpaid_vine_df.distinct().count() 
print(total_number_unpaid)

18019


In [13]:
#the number of 5 star reviews for PAID vine program 
reviews_paid_vine = paid_vine_df.filter("star_rating == 5").count()
print(reviews_paid_vine)

74


In [14]:
#the number of 5 star reviews for UNPAID vine program 
reviews_unpaid_vine = unpaid_vine_df.filter("star_rating == 5").count()
print(reviews_unpaid_vine)

8482


In [17]:
#Percentage of 5 star reviews for PAID vine program 
perc_paid_vine = reviews_paid_vine/total_number_paid * 100
print(perc_paid_vine)

54.41176470588235


In [18]:
#Percentage of 5 star reviews for UNPAID vine program 
perc_unpaid_vine = reviews_unpaid_vine/total_number_unpaid * 100
print(perc_unpaid_vine)

47.072534546867196
